In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/RA2_alpine_lads/ra_joint_predictions')

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from utils.config import Config
import PIL
import json
configuration = Config()

In [3]:
import dataset.combined_joints_dataset as combset
from prediction import joint_damage

In [38]:
individual_model = keras.models.load_model("/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/trained_models/v3/feet_narrowing_reg_rsnaonly_tallandnarrowjoints_sgd.h5", compile = False)
individual_model_2 = keras.models.Model(individual_model.inputs, individual_model.layers[-14].output) 
individual_model_2.trainable = False

In [39]:
input_1 = keras.layers.Input(shape=(224,224,1))
input_2 = keras.layers.Input(shape=(224,224,1))
input_3 = keras.layers.Input(shape=(224,224,1))
input_4 = keras.layers.Input(shape=(224,224,1))
input_5 = keras.layers.Input(shape=(224,224,1))
input_6 = keras.layers.Input(shape=(224,224,1))


In [40]:
out1 = individual_model_2(input_1)
out2 = individual_model_2(input_2)
out3 = individual_model_2(input_3)
out4 = individual_model_2(input_4)
out5 = individual_model_2(input_5)
out6 = individual_model_2(input_6)
pred = keras.layers.Concatenate()([out1,out2,out3,out4,out5,out6,])
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(256, activation = "relu")(pred)
pred = keras.layers.BatchNormalization()(pred)
pred = keras.layers.Dropout(0.5)(pred)
pred = keras.layers.Dense(1)(pred)

In [41]:
combined_model = keras.models.Model(inputs=[input_1,input_2,input_3,input_4,input_5,input_6], outputs=[pred])

In [42]:
combined_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["mae"])

In [43]:
combined_model

In [34]:
train_dataset_class = combset.overall_test_feet(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
train_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv")

In [35]:
val_dataset_class = combset.overall_test_feet(configuration,"/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/fixed")
val_data = train_dataset_class.create_generator(outcomes_source = "/mnt/iusers01/jw01/mdefscs4/ra_challenge/RA_challenge/training_dataset/training.csv",joints_source= "./data/predictions/feet_joint_data_test_v2.csv")

In [ ]:
combined_model.fit_generator(train_data,validation_data = val_data, epochs = 10, steps_per_epoch = 20, validation_steps = 5)

Epoch 1/10
20/20 [==============================] - 167s 8s/step - loss: 15.5015 - mae: 2.5147 - val_loss: 16.7612 - val_mae: 2.8980
Epoch 2/10
20/20 [==============================] - 162s 8s/step - loss: 12.5136 - mae: 2.3147 - val_loss: 48.0001 - val_mae: 5.6098
Epoch 3/10
20/20 [==============================] - 162s 8s/step - loss: 10.6357 - mae: 2.2225 - val_loss: 43.3973 - val_mae: 3.2509
Epoch 4/10
20/20 [==============================] - 158s 8s/step - loss: 16.4037 - mae: 2.6080 - val_loss: 39.6233 - val_mae: 5.4350
Epoch 5/10
20/20 [==============================] - 156s 8s/step - loss: 9.0739 - mae: 2.0507 - val_loss: 7.6222 - val_mae: 1.8837
Epoch 6/10
13/20 [==================>...........] - ETA: 51s - loss: 14.2537 - mae: 2.2164

In [25]:
for layer in combined_model.layers:
    layer.trainable = True

In [45]:
combined_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 224, 224, 1) 0                                            
____________________________________________________________________________________________